In [1]:
import os
import pandas as pd
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, Conv1D, GlobalMaxPooling1D, Concatenate, AdditiveAttention
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import AdditiveAttention
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from utils import eval, eval_training, get_train_test_data, save_for_evaluation, process_train_test_data
import re
import numpy as np
import spacy
from gensim.models import Word2Vec
from transformers import BertTokenizer, TFBertModel


os.environ['TF_NUM_INTEROP_THREADS'] = '10'
os.environ['TF_NUM_INTRAOP_THREADS'] = '10'

# Configure TensorFlow session for multi-threading
tf.config.threading.set_inter_op_parallelism_threads(10)
tf.config.threading.set_intra_op_parallelism_threads(10)
# Ensure TensorFlow is using the Metal backend
gpu = len (tf.config.list_physical_devices ('GPU'))>0
print ("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


# Data preparation

In [4]:
data_label = "data"
predict_label = "label"
balanced = False
labels = None
class_weights = False
sample_weights = False
df_train = pd.read_csv('../data/data_train.csv')
df_valid = pd.read_csv('../data/data_valid.csv')
df_test = pd.read_csv('../data/data_test.csv')

train_texts = df_train[data_label]
valid_texts = df_valid[data_label]
test_texts = df_test[data_label]

all_labels = pd.concat([df_train[predict_label], df_valid[predict_label], df_test[predict_label]])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
# Fit label encoder and return encoded labels as integers
train_labels_enc = label_encoder.transform(df_train[predict_label])
valid_labels_enc = label_encoder.transform(df_valid[predict_label])
test_labels_enc = label_encoder.transform(df_test[predict_label])

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels_enc)
valid_labels = to_categorical(valid_labels_enc)
test_labels = to_categorical(test_labels_enc)

num_classes = len(df_train[predict_label].unique())

In [5]:
from transformers import BertTokenizer, BertModel
import torch
import concurrent.futures
import pickle
import numpy as np

# Function to generate word embeddings for a given batch of texts
def generate_bert_embeddings_batch(texts, tokenizer, model, max_length):
    inputs = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    return embeddings.numpy()

model_name = "GerMedBERT/medbert-512"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to generate embeddings concurrently in batches
def generate_embeddings_concurrently(texts, tokenizer, model, max_length, batch_size=32):
    embeddings = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            futures.append(executor.submit(generate_bert_embeddings_batch, batch_texts, tokenizer, model, max_length))
        
        for future in concurrent.futures.as_completed(futures):
            try:
                batch_embeddings = future.result()
                embeddings.extend(batch_embeddings)
            except Exception as exc:
                print(f'Generated an exception: {exc}')
    return np.array(embeddings)

# Ensure texts are in list format
train_texts = train_texts.tolist()
valid_texts = valid_texts.tolist()
test_texts = test_texts.tolist()

# Define the max length for padding
max_length = 400  # or set to a specific value based on your data
print("Starting")

# Generate embeddings concurrently in batches
bert_train_embeddings = generate_embeddings_concurrently(train_texts, tokenizer, model, max_length)
bert_padded_train_embeddings = np.pad(bert_train_embeddings, ((0,0),(0,max_length-bert_train_embeddings.shape[1]),(0,0)), 'constant')
# Save padded embeddings to files
with open('bert_train.pkl', 'wb') as file:
    pickle.dump(bert_padded_train_embeddings, file)
print("Training done")

bert_valid_embeddings = generate_embeddings_concurrently(valid_texts, tokenizer, model, max_length)
bert_padded_valid_embeddings = np.pad(bert_valid_embeddings, ((0,0),(0,max_length-bert_valid_embeddings.shape[1]),(0,0)), 'constant')
with open('bert_valid.pkl', 'wb') as file:
    pickle.dump(bert_padded_valid_embeddings, file)
print("Valid done")

bert_test_embeddings = generate_embeddings_concurrently(test_texts, tokenizer, model, max_length)
bert_padded_test_embeddings = np.pad(bert_test_embeddings, ((0,0),(0,max_length-bert_test_embeddings.shape[1]),(0,0)), 'constant')
with open('bert_test.pkl', 'wb') as file:
    pickle.dump(bert_padded_test_embeddings, file)
print("Test done")


Some weights of BertModel were not initialized from the model checkpoint at GerMedBERT/medbert-512 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting


: 

In [ ]:
import pickle
with open("bert_train.pkl", "rb") as input_file:
    bert_padded_train_embeddings = pickle.load(input_file)
with open("bert_valid.pkl", "rb") as input_file:
    bert_padded_valid_embeddings = pickle.load(input_file)
with open("bert_test.pkl", "rb") as input_file:
    bert_padded_test_embeddings = pickle.load(input_file)

# Model parameters

In [ ]:
embedding_dim = bert_padded_train_embeddings.shape[-1]
max_length = 400
model_name = "bert"

In [ ]:
from utils import get_standard_parameters

optimizer, loss, metrics, early_stopping_callback, _, lstm_units, epochs, batch_size = get_standard_parameters()

# Model definition and training

In [ ]:
# Define the input layers
input_text = Input(shape=(max_length, embedding_dim), dtype='float32', name='text_input')

# Two LSTM layers
x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(input_text)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(units=lstm_units))(x)

# Output layer
output = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=input_text, outputs=output)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history = model.fit(bert_padded_train_embeddings, train_labels, epochs=epochs, batch_size=batch_size,
                    validation_data=(bert_padded_valid_embeddings, valid_labels), callbacks=[early_stopping_callback])

In [ ]:
save_for_evaluation(model, history, model_name, test_padded, test_labels, label_encoder)

In [ ]:
eval_training(history, metrics)

In [ ]:
eval(model, test_padded, test_labels, label_encoder)